<h1 style="text-align:center;">Data Wrangling</h1>

# Introduction

**Data wrangling**, often also referred to as **data munging**, is the process of cleaning, structuring, and enriching raw data into a desired format for better decision-making. It's a fundamental step in the data preparation process before analysis or processing. Data wrangling involves several tasks and can be quite complex depending on the state of the data and the desired outcome. 

Here are some steps you would follow:

1. **Cleaning**: This involves removing, correcting, or handling corrupted, misformatted, incomplete, or inaccurate data. Common tasks include handling missing values, removing duplicates, and correcting data inconsistencies.
2. **Transforming**: This involves converting data from one format or structure into another. Examples include normalizing and scaling, pivoting tables, or converting data types.
3.  **Enriching**: Enhancing data with additional variables or attributes can make the dataset more useful. This could involve adding data from other sources or creating new derived variables.
4.  **Validating**: Ensuring that the dataset meets certain criteria, often set by predefined rules or schemas. This can involve checking for data consistency, accuracy, and relevance.
5.  **Structuring**: Organizing data in a way that is suitable for the intended purpose. This could involve grouping, sorting, or aggregating data, or reshaping datasets.
6.  **Integrating**: Combining data from multiple sources, which may involve tasks like data alignment, deduplication, and dealing with data source discrepancies.

## Chaining

In this work, we will use a concept called _Chaining_. At its core, chaining is about executing multiple operations in a sequence, where the output of one operation feeds directly into the input of the next.

Think of a factory assembly line where raw materials enter one end and go through several machines, each performing a specific task, before a finished product comes out the other end. At each stage, the output of one machine becomes the input for the next.

In our case, using pandas, chaining mirrors this assembly line concept. Instead of machines, we have methods (functions associated with objects), and instead of raw material, we have data.

One thing we will spend time doing would be to create a new column or alter an old one; and there are several ways we could do this. 

```python
df['new_column'] = value_or_function

df.assign(new_column=value_or_function)
```

Both of the methods above would give us a new column with the name stated or if the column already exists, to alter the contents based off of the values assigned to it. In this walk-through, we will strictly use the `.assign` method most of the time; as this lends itself to the method of chaining well.


In [1]:
import pandas as pd
import datetime as dt

## Datasets: Bike Rentals

We are opting out for the bike rentals dataset from the UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/index.php), a world-famous data warehouse that is free to the public. The author of Hands-On Gradient Boosting with XGBoost and scikit-learn adjusted from the original dataset. We will depend on this book for our walk-through here.

In [2]:
url = 'https://raw.githubusercontent.com/theAfricanQuant/XGBoost4machinelearning/main/data/bike_rentals.csv'

In [3]:
def get_data(url):
    return (
        pd.read_csv(url)
    )

df_bikes = get_data(url)

In [4]:
df_bikes.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1.0,0.0,1.0,0.0,6.0,0.0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1.0,0.0,1.0,0.0,0.0,0.0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1.0,0.0,1.0,0.0,1.0,1.0,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1.0,0.0,1.0,0.0,2.0,1.0,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1.0,0.0,1.0,0.0,3.0,1.0,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


## Understanding the data

In [5]:
df_bikes.describe()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,731.000000,731.000000,730.000000,730.000000,731.000000,731.000000,731.000000,731.000000,730.000000,730.000000,728.000000,726.000000,731.000000,731.000000,731.000000
mean,366.000000,2.496580,0.500000,6.512329,0.028728,2.997264,0.682627,1.395349,0.495587,0.474512,0.627987,0.190476,848.176471,3656.172367,4504.348837
std,211.165812,1.110807,0.500343,3.448303,0.167155,2.004787,0.465773,0.544894,0.183094,0.163017,0.142331,0.077725,686.622488,1560.256377,1937.211452
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.059130,0.079070,0.000000,0.022392,2.000000,20.000000,22.000000
25%,183.500000,2.000000,0.000000,4.000000,0.000000,1.000000,0.000000,1.000000,0.336875,0.337794,0.521562,0.134494,315.500000,2497.000000,3152.000000
50%,366.000000,3.000000,0.500000,7.000000,0.000000,3.000000,1.000000,1.000000,0.499166,0.487364,0.627083,0.180971,713.000000,3662.000000,4548.000000
75%,548.500000,3.000000,1.000000,9.750000,0.000000,5.000000,1.000000,2.000000,0.655625,0.608916,0.730104,0.233218,1096.000000,4776.500000,5956.000000
max,731.000000,4.000000,1.000000,12.000000,1.000000,6.000000,1.000000,3.000000,0.861667,0.840896,0.972500,0.507463,3410.000000,6946.000000,8714.000000


In [6]:
df_bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     731 non-null    int64  
 1   dteday      731 non-null    object 
 2   season      731 non-null    float64
 3   yr          730 non-null    float64
 4   mnth        730 non-null    float64
 5   holiday     731 non-null    float64
 6   weekday     731 non-null    float64
 7   workingday  731 non-null    float64
 8   weathersit  731 non-null    int64  
 9   temp        730 non-null    float64
 10  atemp       730 non-null    float64
 11  hum         728 non-null    float64
 12  windspeed   726 non-null    float64
 13  casual      731 non-null    int64  
 14  registered  731 non-null    int64  
 15  cnt         731 non-null    int64  
dtypes: float64(10), int64(5), object(1)
memory usage: 91.5+ KB


**Correcting null values**

The following code will sum the total number of null values in the dataset. We will chain the methods and functions together horizontally.

In [7]:
def total_nulls(df):
    return (df
         .isna()
         .sum()
         .sum()
        )

total_nulls(df_bikes)

12

We will now create a function that can display the the rows that have null values. In creating a function, we plan to use it over and over again in this project or any other in the future.

In [8]:
def show_nulls(df):
    return (df[df
            .isna()
            .any(axis=1)]
           )

show_nulls(df_bikes)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
56,57,2011-02-26,1.0,0.0,2.0,0.0,6.0,0.0,1,0.282500,0.282192,0.537917,NaN,424,1545,1969
81,82,2011-03-23,2.0,0.0,3.0,0.0,3.0,1.0,2,0.346957,0.337939,0.839565,NaN,203,1918,2121
128,129,2011-05-09,2.0,0.0,5.0,0.0,1.0,1.0,1,0.532500,0.525246,0.588750,NaN,664,3698,4362
129,130,2011-05-10,2.0,0.0,5.0,0.0,2.0,1.0,1,0.532500,0.522721,NaN,0.115671,694,4109,4803
213,214,2011-08-02,3.0,0.0,8.0,0.0,2.0,1.0,1,0.783333,0.707071,NaN,0.205850,801,4044,4845
298,299,2011-10-26,4.0,0.0,10.0,0.0,3.0,1.0,2,0.484167,0.472846,0.720417,NaN,404,3490,3894
388,389,2012-01-24,1.0,1.0,1.0,0.0,2.0,1.0,1,0.342500,0.349108,NaN,0.123767,439,3900,4339
528,529,2012-06-12,2.0,1.0,6.0,0.0,2.0,1.0,2,0.653333,0.597875,0.833333,NaN,477,4495,4972
701,702,2012-12-02,4.0,1.0,12.0,0.0,0.0,0.0,2,NaN,NaN,0.823333,0.124379,892,3757,4649
730,731,2012-12-31,1.0,NaN,NaN,0.0,1.0,0.0,2,0.215833,0.223487,0.577500,0.154846,439,2290,2729


We will replace the null values in the windspeed column with the median. We choose to use the median over the median because the mean tends to guarantee that half the data is greater than the given value and half the data is lower. The mean, by contrast, is vulnerable to outliers. This is the begining of building our wrangle function and we will call it `prep_data`. We will continue to build it step by step until the end of the notebook.

In [9]:
def prep_data(data):
    return (data
            .assign(windspeed = data["windspeed"]
                    .fillna((data["windspeed"]
                             .median())))           
           )

bikes = prep_data(df_bikes)
show_nulls(bikes)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
129,130,2011-05-10,2.0,0.0,5.0,0.0,2.0,1.0,1,0.532500,0.522721,NaN,0.115671,694,4109,4803
213,214,2011-08-02,3.0,0.0,8.0,0.0,2.0,1.0,1,0.783333,0.707071,NaN,0.205850,801,4044,4845
388,389,2012-01-24,1.0,1.0,1.0,0.0,2.0,1.0,1,0.342500,0.349108,NaN,0.123767,439,3900,4339
701,702,2012-12-02,4.0,1.0,12.0,0.0,0.0,0.0,2,NaN,NaN,0.823333,0.124379,892,3757,4649
730,731,2012-12-31,1.0,NaN,NaN,0.0,1.0,0.0,2,0.215833,0.223487,0.577500,0.154846,439,2290,2729


For the 'windspeed' column, the null values may be replaced with the median value

In [10]:
bikes.iloc[[56, 81]]

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
56,57,2011-02-26,1.0,0.0,2.0,0.0,6.0,0.0,1,0.282500,0.282192,0.537917,0.180971,424,1545,1969
81,82,2011-03-23,2.0,0.0,3.0,0.0,3.0,1.0,2,0.346957,0.337939,0.839565,0.180971,203,1918,2121


It's possible to get more nuanced when correcting null values by using a `groupby`.

A `groupby` organizes rows by shared values. Since there are four shared seasons spread out among the rows, a `groupby` of seasons results in a total of four rows, one for each season. But each season comes from many different rows with different values. We need a way to combine, or aggregate, the values. Choices for the aggregate include `.sum()`, .`count()`, .`mean()`, and .`median()`. 

Grouping our dataframe by season with the `.median(numeric_only=True)` aggregate is achieved as follows:

In [11]:
bikes.groupby(['season']).median(numeric_only=True)

,instant,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
season,,,,,,,,,,,,,,
1.0,366.0,0.5,2.0,0.0,3.0,1.0,1.0,0.285833,0.282821,0.543750,0.202750,218.0,1867.0,2209.0
2.0,308.5,0.5,5.0,0.0,3.0,1.0,1.0,0.562083,0.538212,0.646667,0.191546,867.0,3844.0,4941.5
3.0,401.5,0.5,8.0,0.0,3.0,1.0,1.0,0.714583,0.656575,0.635833,0.165115,1050.5,4110.5,5353.5
4.0,493.0,0.5,11.0,0.0,3.0,1.0,1.0,0.410000,0.409708,0.661042,0.167918,544.5,3815.0,4634.5


To correct the null values in the `'hum'` column (humidity), we can take the median humidity by season.
```python
bikes['hum'] = bikes['hum'].fillna()
```
The code that goes inside fillna is the desired values. The values obtained from groupby require the transform method as follows:

```python
bikes.groupby('season')['hum'].transform('median')
```
Bringing everything together:

```python
bikes['hum'] = (bikes['hum']
                   .fillna(bikes.groupby('season')['hum']
                           .transform('median'))
                  )
```

However, to implement it we are going to use the method of chaining and the `.assign` method in our code.

In [12]:
def prep_data(data):
    return (data
            .assign(windspeed = data["windspeed"]
                    .fillna((data["windspeed"]
                             .median())),
                    hum = (data['hum']
                   .fillna(data.groupby('season')['hum']
                           .transform('median'))
                  )
                   )           
           )

bikes = prep_data(df_bikes)
show_nulls(bikes)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
701,702,2012-12-02,4.0,1.0,12.0,0.0,0.0,0.0,2,NaN,NaN,0.823333,0.124379,892,3757,4649
730,731,2012-12-31,1.0,NaN,NaN,0.0,1.0,0.0,2,0.215833,0.223487,0.577500,0.154846,439,2290,2729


from the above we can see that the column `'hum'` has been taken off the those with `'nan'` values.

In [13]:
bikes.iloc[[129, 213, 388]]

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
129,130,2011-05-10,2.0,0.0,5.0,0.0,2.0,1.0,1,0.532500,0.522721,0.646667,0.115671,694,4109,4803
213,214,2011-08-02,3.0,0.0,8.0,0.0,2.0,1.0,1,0.783333,0.707071,0.635833,0.205850,801,4044,4845
388,389,2012-01-24,1.0,1.0,1.0,0.0,2.0,1.0,1,0.342500,0.349108,0.543750,0.123767,439,3900,4339


**Obtaining the median/mean from specific rows**

In some cases, it may be advantageous to replace null values with data from specific rows.

When correcting temperature, aside from consulting historical records, taking the mean temperature of the day before and the day after should give a good estimate. 

To find null values of the 'temp' column, enter the following code:

In [14]:
bikes[bikes['temp'].isna()]

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
701,702,2012-12-02,4.0,1.0,12.0,0.0,0.0,0.0,2,NaN,NaN,0.823333,0.124379,892,3757,4649


Index `701` contains null values.

We will now find the mean temperature of the day before and the day after the 701 index, using the some steps:

1. Let us sum the temperatures in rows `700` and `702` and divide by `2` for both the `'temp'` and `'atemp'` columns

In [15]:
def mean_vals(df, idx1, idx2, col):
    return (
        (df.iloc[idx1][col] + 
        df.iloc[idx2][col])/2
    )

mean_vals(bikes, 700, 702, 'atemp')

0.38634999999999997

In [16]:
def prep_data(data):
    return (data
            .assign(windspeed = data["windspeed"]
                    .fillna((data["windspeed"]
                             .median())),
                    hum = (data['hum']
                   .fillna(data.groupby('season')['hum']
                           .transform('median'))),
                    temp = (data['temp']
                            .fillna(mean_vals(data, 700, 702, 'temp'))),
                    atemp = (data['atemp']
                            .fillna(mean_vals(data, 700, 702, 'atemp')))
                   )           
           )

bikes = prep_data(df_bikes)
show_nulls(bikes)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
730,731,2012-12-31,1.0,NaN,NaN,0.0,1.0,0.0,2,0.215833,0.223487,0.5775,0.154846,439,2290,2729


**Extrapolate dates**

`'dteday'` is meant to be a date column but the `.info` we ran earlier revealed to us that it was an object or a string. Date objects such as years and months must be extrapolated from `datetime` types. Lets convert the column to a `datetime`.

In [17]:
def prep_data(data):
    return (data
            .assign(windspeed = data["windspeed"]
                    .fillna((data["windspeed"]
                             .median())),
                    hum = (data['hum']
                   .fillna(data.groupby('season')['hum']
                           .transform('median'))),
                    temp = (data['temp']
                            .fillna(mean_vals(data, 700, 702, 'temp'))),
                    atemp = (data['atemp']
                            .fillna(mean_vals(data, 700, 702, 'atemp'))),
                    dteday = pd.to_datetime(data['dteday'])
                   )           
           )

bikes = (prep_data(df_bikes)
        .info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   instant     731 non-null    int64         
 1   dteday      731 non-null    datetime64[ns]
 2   season      731 non-null    float64       
 3   yr          730 non-null    float64       
 4   mnth        730 non-null    float64       
 5   holiday     731 non-null    float64       
 6   weekday     731 non-null    float64       
 7   workingday  731 non-null    float64       
 8   weathersit  731 non-null    int64         
 9   temp        731 non-null    float64       
 10  atemp       731 non-null    float64       
 11  hum         731 non-null    float64       
 12  windspeed   731 non-null    float64       
 13  casual      731 non-null    int64         
 14  registered  731 non-null    int64         
 15  cnt         731 non-null    int64         
dtypes: datetime64[ns](1), floa

We have imported the `'datetime'` library above. 

We will convert the `'mnth'` column to the correct months extrpolated from the `'dteday'` column.

In [18]:
def prep_data(data):
    return (data
            .assign(windspeed = data["windspeed"]
                    .fillna((data["windspeed"]
                             .median())),
                    hum = (data['hum']
                   .fillna(data.groupby('season')['hum']
                           .transform('median'))),
                    temp = (data['temp']
                            .fillna(mean_vals(data, 700, 702, 'temp'))),
                    atemp = (data['atemp']
                            .fillna(mean_vals(data, 700, 702, 'atemp'))),
                    dteday = pd.to_datetime(data['dteday']),
                    mnth = lambda x: x['dteday'].dt.month
                   )           
           )

bikes = prep_data(df_bikes)
show_nulls(bikes)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
730,731,2012-12-31,1.0,NaN,12,0.0,1.0,0.0,2,0.215833,0.223487,0.5775,0.154846,439,2290,2729


lets us check the last 5 values of the dataset we have worked on so far.

In [19]:
bikes.tail(5)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
726,727,2012-12-27,1.0,1.0,12,0.0,4.0,1.0,2,0.254167,0.226642,0.652917,0.350133,247,1867,2114
727,728,2012-12-28,1.0,1.0,12,0.0,5.0,1.0,2,0.253333,0.255046,0.590000,0.155471,644,2451,3095
728,729,2012-12-29,1.0,1.0,12,0.0,6.0,0.0,2,0.253333,0.242400,0.752917,0.124383,159,1182,1341
729,730,2012-12-30,1.0,1.0,12,0.0,0.0,0.0,1,0.255833,0.231700,0.483333,0.350754,364,1432,1796
730,731,2012-12-31,1.0,NaN,12,0.0,1.0,0.0,2,0.215833,0.223487,0.577500,0.154846,439,2290,2729


We can see that even though the year value on the `'dteday'` column has 2012 all through, the value on the `'yr'` column is `1.0`. It probably means that the values have been normalized, probably between 0 & 1. I think this was done because normalized data is often more efficient due to the fact that machine learning weights do not have to adjust for different ranges.

We will just use the forward fill for the null values here since the row with the null value is in the same month with the preceding row.

```python
data['yr'].ffill()
```

or 

```python
data['yr'].fillna(method='ffill')
```

In [20]:
def prep_data(data):
    return (data
            .assign(windspeed = data["windspeed"]
                    .fillna((data["windspeed"]
                             .median())),
                    hum = (data['hum']
                   .fillna(data.groupby('season')['hum']
                           .transform('median'))),
                    temp = (data['temp']
                            .fillna(mean_vals(data, 700, 702, 'temp'))),
                    atemp = (data['atemp']
                            .fillna(mean_vals(data, 700, 702, 'atemp'))),
                    dteday = pd.to_datetime(data['dteday']),
                    mnth = lambda x: x['dteday'].dt.month,
                    yr = data['yr'].ffill()
                   )           
           )

bikes = prep_data(df_bikes)
show_nulls(bikes)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt


There are no more null values in our dataset.

In [21]:
total_nulls(bikes)

0

In [22]:
bikes.tail()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
726,727,2012-12-27,1.0,1.0,12,0.0,4.0,1.0,2,0.254167,0.226642,0.652917,0.350133,247,1867,2114
727,728,2012-12-28,1.0,1.0,12,0.0,5.0,1.0,2,0.253333,0.255046,0.590000,0.155471,644,2451,3095
728,729,2012-12-29,1.0,1.0,12,0.0,6.0,0.0,2,0.253333,0.242400,0.752917,0.124383,159,1182,1341
729,730,2012-12-30,1.0,1.0,12,0.0,0.0,0.0,1,0.255833,0.231700,0.483333,0.350754,364,1432,1796
730,731,2012-12-31,1.0,1.0,12,0.0,1.0,0.0,2,0.215833,0.223487,0.577500,0.154846,439,2290,2729


**Deleting non-numerical columns**

For machine learning, all data columns should be numerical. According to `df.info()`, the only column that is not numerical is `'dteday'`. Furthermore, it's redundant since all date information exists in other columns.

In [23]:
def prep_data(data):
    return (data
            .assign(windspeed = data["windspeed"]
                    .fillna((data["windspeed"]
                             .median())),
                    hum = (data['hum']
                   .fillna(data.groupby('season')['hum']
                           .transform('median'))),
                    temp = (data['temp']
                            .fillna(mean_vals(data, 700, 702, 'temp'))),
                    atemp = (data['atemp']
                            .fillna(mean_vals(data, 700, 702, 'atemp'))),
                    dteday = pd.to_datetime(data['dteday']),
                    mnth = lambda x: x['dteday'].dt.month,
                    yr = data['yr'].ffill()
                   )
            .drop('dteday', axis=1)
           )

bikes = prep_data(df_bikes)
bikes.head()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,1.0,0.0,1,0.0,6.0,0.0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,1.0,0.0,1,0.0,0.0,0.0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,1.0,0.0,1,0.0,1.0,1.0,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,1.0,0.0,1,0.0,2.0,1.0,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,1.0,0.0,1,0.0,3.0,1.0,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


### Final Code

The next thing we will do is to create a module with the functions that we have created so we could always call them whenever we need them.

In [24]:
%%writefile data_wrangle.py

import pandas as pd
import datetime as dt


def show_nulls(df):
    return (df[df
            .isna()
            .any(axis=1)]
           )
    
def total_nulls(df):
    return (df
         .isna()
         .sum()
         .sum()
        )

def get_data(url):
    return (
        pd.read_csv(url)
    )

def mean_vals(df, idx1, idx2, col):
    return (
        (df.iloc[idx1][col] + 
        df.iloc[idx2][col])/2
    )

def prep_data(data):
    return (data
            .assign(windspeed = data["windspeed"]
                    .fillna((data["windspeed"]
                             .median())),
                    hum = (data['hum']
                   .fillna(data.groupby('season')['hum']
                           .transform('median'))),
                    temp = (data['temp']
                            .fillna(mean_vals(data, 700, 702, 'temp'))),
                    atemp = (data['atemp']
                            .fillna(mean_vals(data, 700, 702, 'atemp'))),
                    dteday = pd.to_datetime(data['dteday']),
                    mnth = lambda x: x['dteday'].dt.month,
                    yr = data['yr'].ffill()
                   )
            .drop('dteday', axis=1)
           )

Writing data_wrangle.py
